In [1]:
# =======================
# Imports & Configuration
# =======================
import warnings
warnings.filterwarnings('ignore')

# Core libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn: Preprocessing, Modeling, Evaluation, Feature Selection
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, VarianceThreshold

# Statistical tools
from scipy import stats
from scipy.stats import chi2_contingency

# Matplotlib & seaborn configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9


## Step 0: Data Loading and Inspection

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/spaceship_titanic_train.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
# Let's print some descriptive statistics for all the numeric features.

df.describe().T
# This gives that it is highly right-skewed for all numeric features apart from age
# The age distribution is right-skewed. The majority of passengers are young adults (20–30).
# The median is close to the mean, the skew isn't too extreme.

,count,mean,std,min,25%,50%,75%,max
Age,8514.0,28.827930,14.489021,0.0,19.0,27.0,38.0,79.0
RoomService,8512.0,224.687617,666.717663,0.0,0.0,0.0,47.0,14327.0
FoodCourt,8510.0,458.077203,1611.489240,0.0,0.0,0.0,76.0,29813.0
ShoppingMall,8485.0,173.729169,604.696458,0.0,0.0,0.0,27.0,23492.0
Spa,8510.0,311.138778,1136.705535,0.0,0.0,0.0,59.0,22408.0
VRDeck,8505.0,304.854791,1145.717189,0.0,0.0,0.0,46.0,24133.0


In [5]:
# Let's print some descriptive statistics for all the numeric features.

df.describe().T# What is the number of unique values in all the categorical features? And what is
# the value with the highest frequency?

df.describe(include=object).T
# can frop 'Name' feature
# looking at the dataset from cabin, can see whether or not passenger is solo or in a group.

,count,unique,top,freq
PassengerId,8693,8693,0001_01,1
HomePlanet,8492,3,Earth,4602
CryoSleep,8476,2,False,5439
Cabin,8494,6560,G/734/S,8
Destination,8511,3,TRAPPIST-1e,5915
VIP,8490,2,False,8291
Name,8493,8473,Gollux Reedall,2


In [6]:
# How much missing data is in each feature?

df.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
# For convienience, let's save the names of all numeric features to a list,
# and the names of all categorical features to another list.

numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

categorical_features = ['HomePlanet', 'VIP', 'CryoSleep', 'Destination', 'Cabin', 'Name']

In [8]:
# ================================================================
# SPACESHIP TITANIC: RESCUE MISSION - ADVANCED EDA & FEATURE ENGINEERING
# ================================================================
# Mission: Extract every signal from damaged ship logs to save more lives
# Objective: Build AI-powered triage engine for passenger rescue prediction
# Target: Every 1% accuracy improvement = hundreds more lives saved

print("🚀 SPACESHIP TITANIC RESCUE MISSION INITIATED")
print("=" * 60)

# ================================================================
# PHASE 1: INTELLIGENCE GATHERING - LOAD & INITIAL INSPECTION
# ================================================================

def load_and_inspect_data():
    """Load data and perform initial intelligence gathering"""
    print("\n📊 PHASE 1: INTELLIGENCE GATHERING")
    print("-" * 40)

    # Load training data
    df = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/spaceship_titanic_train.csv")

    print(f"🔍 Mission Log Analysis:")
    print(f"   - Total passengers in manifest: {len(df):,}")
    print(f"   - Data integrity: {df.shape[1]} features recorded")
    print(f"   - Missing data assessment needed...")

    return df

def comprehensive_data_quality_analysis(df):
    """Deep dive into data quality and patterns"""
    print("\n🔬 COMPREHENSIVE DATA QUALITY ANALYSIS")
    print("-" * 45)

    # Basic info
    print(f"Dataset Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Missing data analysis
    missing_analysis = pd.DataFrame({
        'Column': df.columns,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
        'Data_Type': df.dtypes
    }).sort_values('Missing_Percentage', ascending=False)

    print("\n📋 MISSING DATA INTELLIGENCE REPORT:")
    print(missing_analysis[missing_analysis['Missing_Count'] > 0])

    # Class distribution analysis
    print("\n⚖️ TARGET DISTRIBUTION ANALYSIS:")
    target_dist = df['Transported'].value_counts()
    print(f"Transported: {target_dist[True]:,} ({target_dist[True]/len(df)*100:.1f}%)")
    print(f"Not Transported: {target_dist[False]:,} ({target_dist[False]/len(df)*100:.1f}%)")

    # Data types summary
    print(f"\n📊 DATA TYPES SUMMARY:")
    dtype_summary = df.dtypes.value_counts()
    for dtype, count in dtype_summary.items():
        print(f"   {dtype}: {count} features")

    return missing_analysis


try:
    # Execute the complete EDA pipeline
    print("🚀 Starting Spaceship Titanic Rescue Mission Analysis...")

    # Phase 1: Load and inspect data
    df = load_and_inspect_data()

    # Phase 2: Comprehensive data quality analysis
    missing_analysis = comprehensive_data_quality_analysis(df)

    # Mission completion summary
    print("\n" + "=" * 60)
    print("🎉 RESCUE MISSION EDA COMPLETE!")
    print("=" * 60)
    print(f"✅ Analyzed {len(df):,} passenger records")
    print(f"✅ Generated comprehensive visualizations")
    print(f"✅ Identified key rescue patterns")
    print("\n🚨 Ready for model development phase!")

except Exception as e:
    print(f"❌ Mission encountered error: {str(e)}")
    print("🔧 Check data source and dependencies")

🚀 SPACESHIP TITANIC RESCUE MISSION INITIATED
🚀 Starting Spaceship Titanic Rescue Mission Analysis...

📊 PHASE 1: INTELLIGENCE GATHERING
----------------------------------------
🔍 Mission Log Analysis:
   - Total passengers in manifest: 8,693
   - Data integrity: 14 features recorded
   - Missing data assessment needed...

🔬 COMPREHENSIVE DATA QUALITY ANALYSIS
---------------------------------------------
Dataset Shape: (8693, 14)
Memory Usage: 3.65 MB

📋 MISSING DATA INTELLIGENCE REPORT:
                    Column  Missing_Count  Missing_Percentage Data_Type
CryoSleep        CryoSleep            217                2.50    object
ShoppingMall  ShoppingMall            208                2.39   float64
VIP                    VIP            203                2.34    object
HomePlanet      HomePlanet            201                2.31    object
Name                  Name            200                2.30    object
Cabin                Cabin            199                2.29    object
VRD

In [9]:
# ================================================================
# SPACESHIP TITANIC: FIXED FEATURE ENGINEERING PIPELINE
# ================================================================
# FIXES: Target leakage, overfitting, train/test inconsistencies
# REMOVES: All target-derived features that cause 96% train vs 70% test gap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold, SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import KFold
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def basic_missing_value_imputation(df):
    """Basic imputation before feature engineering"""
    print("🔧 Basic Missing Value Imputation...")

    # Age: Simple imputation first (will be improved later)
    df['Age'] = df['Age'].fillna(df['Age'].median())

    # Spending: Logical assumption - NaN means no spending
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in spending_cols:
        df[col] = df[col].fillna(0)

    # CryoSleep/VIP: Conservative assumption - NaN means False
    df['CryoSleep'] = df['CryoSleep'].fillna(False)
    df['VIP'] = df['VIP'].fillna(False)

    # Categorical: Mode imputation with domain knowledge
    df['HomePlanet'] = df['HomePlanet'].fillna(df['HomePlanet'].mode()[0])
    df['Destination'] = df['Destination'].fillna(df['Destination'].mode()[0])

    # Cabin: Create 'Unknown' category for missing values
    df['Cabin'] = df['Cabin'].fillna('Unknown/0/U')

    print(f"   → Remaining missing values: {df.isnull().sum().sum()}")
    return df

def advanced_missing_value_imputation(df):
    """Advanced hierarchical imputation after group features are created"""
    print("🔧 Advanced Missing Value Imputation...")

    # Now we can do hierarchical imputation since PassengerGroup exists
    if 'PassengerGroup' in df.columns:
        # Age: Hierarchical imputation (Group → Planet → Global)
        df['Age'] = df.groupby('PassengerGroup')['Age'].transform(lambda x: x.fillna(x.median()))
        df['Age'] = df.groupby('HomePlanet')['Age'].transform(lambda x: x.fillna(x.median()))
        df['Age'] = df['Age'].fillna(df['Age'].median())
        print("   → Applied hierarchical age imputation")

    return df

def detect_and_handle_outliers(df):
    """Statistical outlier detection and treatment"""
    print("📊 Outlier Detection & Treatment...")

    # Age outliers (beyond reasonable human lifespan)
    age_outliers = (df['Age'] > 100).sum()
    df['Age_Capped'] = df['Age'].clip(upper=100)

    # Spending outliers using IQR method
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

    for col in spending_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        df[f'{col}_Capped'] = df[col].clip(lower=lower_bound, upper=upper_bound)

        if outliers_count > 0:
            print(f"   → {col}: {outliers_count} outliers capped")

    # Create outlier flags (useful features)
    df['HasAgeOutlier'] = (df['Age'] > 100).astype(int)
    df['HasSpendingOutlier'] = ((df['RoomService'] > df['RoomService'].quantile(0.99)) |
                                (df['FoodCourt'] > df['FoodCourt'].quantile(0.99)) |
                                (df['ShoppingMall'] > df['ShoppingMall'].quantile(0.99)) |
                                (df['Spa'] > df['Spa'].quantile(0.99)) |
                                (df['VRDeck'] > df['VRDeck'].quantile(0.99))).astype(int)

    print(f"   → Age outliers: {age_outliers}")
    print(f"   → Spending outliers flagged: {df['HasSpendingOutlier'].sum()}")

    return df

def engineer_passenger_groups(df):
    """Extract passenger group features - FIXED: No target leakage"""
    print("🔍 Engineering Passenger Group Features...")

    # Parse PassengerId: format is "group_member" (e.g., "0001_01")
    df['PassengerGroup'] = df['PassengerId'].str.extract('(\d+)_').astype(int)
    df['GroupMember'] = df['PassengerId'].str.extract('_(\d+)').astype(int)

    # Calculate group sizes
    group_counts = df['PassengerGroup'].value_counts()
    df['GroupSize'] = df['PassengerGroup'].map(group_counts)

    # 🚨 REMOVED: GroupSurvivalRate - This was using target variable!
    # Instead, use group characteristics that don't leak target info

    # Solo travelers vs groups
    df['IsSolo'] = (df['GroupSize'] == 1).astype(int)
    df['IsLargeGroup'] = (df['GroupSize'] >= 5).astype(int)
    df['IsMediumGroup'] = ((df['GroupSize'] >= 2) & (df['GroupSize'] <= 4)).astype(int)

    # Group spending patterns - fix the syntax
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    group_total_spend = df.groupby('PassengerGroup')[spending_cols].sum().sum(axis=1)
    df['GroupTotalSpend'] = df['PassengerGroup'].map(group_total_spend)

    # Group age patterns (safe features)
    group_age_stats = df.groupby('PassengerGroup')['Age'].agg(['mean', 'std', 'min', 'max'])
    df['GroupAgeMean'] = df['PassengerGroup'].map(group_age_stats['mean'])
    df['GroupAgeStd'] = df['PassengerGroup'].map(group_age_stats['std']).fillna(0)
    df['GroupAgeRange'] = df['PassengerGroup'].map(group_age_stats['max'] - group_age_stats['min'])

    print(f"   → Group sizes: {df['GroupSize'].min()}-{df['GroupSize'].max()}")
    print(f"   → Solo travelers: {df['IsSolo'].sum():,} ({df['IsSolo'].mean()*100:.1f}%)")
    print("   ✅ FIXED: Removed GroupSurvivalRate (target leakage)")

    return df

def engineer_cabin_features(df):
    """Extract cabin location features - key predictor from EDA"""
    print("🏢 Engineering Cabin Features...")

    # Parse cabin: format is "deck/num/side" (e.g., "B/0/P")
    cabin_split = df['Cabin'].str.split('/', expand=True)
    df['CabinDeck'] = cabin_split[0]
    df['CabinNum'] = pd.to_numeric(cabin_split[1], errors='coerce')
    df['CabinSide'] = cabin_split[2]

    # EDA showed deck importance - encode numerically
    deck_order = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8, 'Unknown': 0}
    df['DeckLevel'] = df['CabinDeck'].map(deck_order)

    # Side preference (Starboard showed better survival in EDA)
    df['IsStarboard'] = (df['CabinSide'] == 'S').astype(int)
    df['IsPort'] = (df['CabinSide'] == 'P').astype(int)
    df['IsUnknownSide'] = (df['CabinSide'] == 'U').astype(int)

    # Cabin number features
    df['CabinNum'] = df['CabinNum'].fillna(0)
    df['CabinNumQuartile'] = pd.qcut(df['CabinNum'].replace(0, np.nan), q=4, labels=[1,2,3,4]).astype(float)
    df['CabinNumQuartile'] = df['CabinNumQuartile'].fillna(0)

    # Cabin luxury indicators
    df['IsLuxuryCabin'] = (df['CabinNumQuartile'] == 4).astype(int)
    df['IsStandardCabin'] = (df['CabinNumQuartile'].isin([2, 3])).astype(int)

    # Missing cabin indicates special case
    df['HasCabin'] = (df['Cabin'] != 'Unknown/0/U').astype(int)

    print(f"   → Decks found: {sorted(df['CabinDeck'].unique())}")
    print(f"   → Starboard preference: {df['IsStarboard'].mean()*100:.1f}%")

    return df

def engineer_spending_features(df):
    """Create spending features - major insight from EDA showing inverse relationship"""
    print("💰 Engineering Spending Features...")

    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    capped_spending_cols = [f'{col}_Capped' for col in spending_cols]

    # Use capped versions for feature engineering
    for orig, capped in zip(spending_cols, capped_spending_cols):
        if capped in df.columns:
            df[orig] = df[capped]

    # Total spending (key finding: negative correlation with survival)
    df['TotalSpend'] = df[spending_cols].sum(axis=1)

    # Spending categories based on EDA patterns
    df['LuxurySpend'] = df['Spa'] + df['VRDeck']
    df['FoodSpend'] = df['RoomService'] + df['FoodCourt']
    df['ShoppingSpend'] = df['ShoppingMall']

    # Binary spending indicators
    df['IsSpender'] = (df['TotalSpend'] > 0).astype(int)
    df['IsHighSpender'] = (df['TotalSpend'] > df['TotalSpend'].quantile(0.75)).astype(int)
    df['IsLowSpender'] = (df['TotalSpend'] <= df['TotalSpend'].quantile(0.25)).astype(int)
    df['UsesLuxury'] = (df['LuxurySpend'] > 0).astype(int)

    # Log transformations for skewed distributions
    df['TotalSpend_Log'] = np.log1p(df['TotalSpend'])
    df['LuxurySpend_Log'] = np.log1p(df['LuxurySpend'])
    df['FoodSpend_Log'] = np.log1p(df['FoodSpend'])

    # Square root transformations (alternative for skewed data)
    df['TotalSpend_Sqrt'] = np.sqrt(df['TotalSpend'])

    # Spending diversity (how many services used)
    df['SpendDiversity'] = (df[spending_cols] > 0).sum(axis=1)

    # Group spending patterns (safe features)
    df['SpendPerGroupMember'] = df['TotalSpend'] / df['GroupSize']
    df['SpendRatioInGroup'] = df['TotalSpend'] / (df['GroupTotalSpend'] + 1)

    # Spending percentiles within group (safe ranking feature)
    df['SpendPercentileInGroup'] = df.groupby('PassengerGroup')['TotalSpend'].rank(pct=True)

    print(f"   → Non-spenders: {(df['TotalSpend']==0).sum():,} ({(df['TotalSpend']==0).mean()*100:.1f}%)")
    print(f"   → High spenders: {df['IsHighSpender'].sum():,} ({df['IsHighSpender'].mean()*100:.1f}%)")

    return df

def engineer_demographic_features(df):
    """Create demographic features - FIXED: No target leakage"""
    print("👤 Engineering Demographic Features...")

    # Use capped age
    df['Age'] = df['Age_Capped']

    # Age categories based on EDA survival patterns
    df['IsChild'] = (df['Age'] < 13).astype(int)  # Children showed high survival
    df['IsTeen'] = ((df['Age'] >= 13) & (df['Age'] < 18)).astype(int)
    df['IsYoungAdult'] = ((df['Age'] >= 18) & (df['Age'] < 35)).astype(int)
    df['IsMiddleAged'] = ((df['Age'] >= 35) & (df['Age'] < 60)).astype(int)
    df['IsElderly'] = (df['Age'] >= 60).astype(int)

    # Age transformations for non-linear relationships
    df['Age_Squared'] = df['Age'] ** 2
    df['Age_Log'] = np.log1p(df['Age'])
    df['Age_Sqrt'] = np.sqrt(df['Age'])

    # CryoSleep (major predictor from EDA - 82% survival rate)
    df['CryoSleep'] = df['CryoSleep'].astype(int)

    # VIP status (showed negative effect in EDA)
    df['VIP'] = df['VIP'].astype(int)

    # 🚨 REMOVED: All survival rate features that used target variable
    # - PlanetSurvivalRate
    # - DestSurvivalRate
    # - PlanetDestSurvivalRate

    # Instead, use frequency-based features (safe)
    planet_counts = df['HomePlanet'].value_counts()
    dest_counts = df['Destination'].value_counts()

    df['PlanetFrequency'] = df['HomePlanet'].map(planet_counts)
    df['DestFrequency'] = df['Destination'].map(dest_counts)

    # Planet-destination combination frequency (safe)
    planet_dest_counts = df.groupby(['HomePlanet', 'Destination']).size()
    df['PlanetDestFrequency'] = df.set_index(['HomePlanet', 'Destination']).index.map(planet_dest_counts)

    print(f"   → Children: {df['IsChild'].sum():,} ({df['IsChild'].mean()*100:.1f}%)")
    print(f"   → CryoSleep users: {df['CryoSleep'].sum():,} ({df['CryoSleep'].mean()*100:.1f}%)")
    print(f"   → VIP passengers: {df['VIP'].sum():,} ({df['VIP'].mean()*100:.1f}%)")
    print("   ✅ FIXED: Removed survival rate features (target leakage)")

    return df

def engineer_family_features(df):
    """Extract family relationships from names - FIXED: No target leakage"""
    print("👨‍👩‍👧‍👦 Engineering Family Features...")

    # Extract first and last names
    df['FirstName'] = df['Name'].str.split().str[0]
    df['LastName'] = df['Name'].str.split().str[-1]

    # Family size based on last name
    family_counts = df['LastName'].value_counts()
    df['FamilySize'] = df['LastName'].map(family_counts)

    # 🚨 REMOVED: FamilySurvivalRate - This was using target variable!

    # Family categories (safe features)
    df['IsLargeFamily'] = (df['FamilySize'] >= 4).astype(int)
    df['IsMediumFamily'] = (df['FamilySize'] == 2).astype(int)
    df['IsSingleFamily'] = (df['FamilySize'] == 1).astype(int)

    # Family vs group relationship (safe features)
    df['FamilyGroupRatio'] = df['FamilySize'] / df['GroupSize']
    df['IsFamilyGroup'] = (df['FamilySize'] == df['GroupSize']).astype(int)

    # Family name rarity (safe feature)
    df['FamilyNameRarity'] = 1 / df['FamilySize']  # Rare names = higher values

    print(f"   → Family sizes: {df['FamilySize'].min()}-{df['FamilySize'].max()}")
    print(f"   → Large families: {df['IsLargeFamily'].sum():,} ({df['IsLargeFamily'].mean()*100:.1f}%)")
    print("   ✅ FIXED: Removed FamilySurvivalRate (target leakage)")

    return df

def create_safe_interaction_features(df):
    """Create interaction features - SAFE VERSION without target leakage"""
    print("🔗 Creating Safe Interaction Features...")

    # Key interactions from EDA (safe versions)
    df['Age_Spending_Interaction'] = df['Age'] * df['TotalSpend_Log']
    df['Age_Group_Interaction'] = df['Age'] * df['GroupSize']
    df['Deck_Side_Interaction'] = df['DeckLevel'] * df['IsStarboard']
    df['Group_Spending_Interaction'] = df['GroupSize'] * df['TotalSpend_Log']

    # CryoSleep interactions (major predictor)
    df['Cryo_Age'] = df['CryoSleep'] * df['Age']
    df['Cryo_Deck'] = df['CryoSleep'] * df['DeckLevel']
    df['Cryo_VIP'] = df['CryoSleep'] * df['VIP']
    df['Cryo_Spending'] = df['CryoSleep'] * df['TotalSpend_Log']
    df['Cryo_Group'] = df['CryoSleep'] * df['GroupSize']

    # VIP interactions
    df['VIP_Spending'] = df['VIP'] * df['TotalSpend_Log']
    df['VIP_Age'] = df['VIP'] * df['Age']
    df['VIP_Deck'] = df['VIP'] * df['DeckLevel']

    # Age-based interactions
    df['Child_Cryo'] = df['IsChild'] * df['CryoSleep']
    df['Child_VIP'] = df['IsChild'] * df['VIP']
    df['Elderly_Cryo'] = df['IsElderly'] * df['CryoSleep']

    # Reduced complexity interactions (prevent overfitting)
    df['Age_Deck'] = df['Age'] * df['DeckLevel']
    df['Spending_Deck'] = df['TotalSpend_Log'] * df['DeckLevel']

    # Family-group interactions
    df['Family_Group_Size'] = df['FamilySize'] * df['GroupSize']

    print(f"   → Created {16} safe interaction features")
    print("   ✅ REMOVED: Complex 3-way interactions to reduce overfitting")

    return df

def safe_categorical_encoding(df):
    """Encode categorical variables - SAFE VERSION without target leakage"""
    print("🔤 Safe Categorical Encoding...")

    # 🚨 REMOVED: Target encoding - This was the major source of leakage!
    # Target encoding uses the target variable to encode categories

    # Frequency encoding for categorical features (safe)
    freq_encode_features = ['HomePlanet', 'Destination', 'CabinDeck', 'FirstName', 'LastName']

    for feature in freq_encode_features:
        if feature in df.columns:
            freq_map = df[feature].value_counts().to_dict()
            df[f'{feature}_Frequency'] = df[feature].map(freq_map)

    # One-hot encoding for low-cardinality features (safe)
    onehot_features = ['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide']

    for feature in onehot_features:
        if feature in df.columns:
            dummies = pd.get_dummies(df[feature], prefix=feature, drop_first=True)
            df = pd.concat([df, dummies], axis=1)

    # Name length features (safe)
    df['FirstNameLength'] = df['FirstName'].str.len()
    df['LastNameLength'] = df['LastName'].str.len()
    df['FullNameLength'] = df['FirstNameLength'] + df['LastNameLength']

    # Name patterns (safe)
    df['HasCommonFirstName'] = df['FirstName_Frequency'] > 10
    df['HasRareLastName'] = df['LastName_Frequency'] <= 2

    print(f"   → Frequency encoded: {len(freq_encode_features)} features")
    print(f"   → One-hot encoded: {len(onehot_features)} features")
    print("   ✅ REMOVED: Target encoding (major source of leakage)")

    return df

def create_final_feature_set(df):
    """Create final clean feature set for modeling"""
    print("🎯 Creating Final Feature Set...")

    # Restore original PassengerId from backup
    if '_PassengerId_Original' in df.columns:
        df['PassengerId'] = df['_PassengerId_Original']
        df = df.drop(columns=['_PassengerId_Original'])

    # Remove identifier and text columns
    drop_cols = [
        'Name', 'FirstName', 'LastName', 'Cabin',
        'HomePlanet', 'Destination', 'CabinDeck', 'CabinSide',
        # Remove capped versions as we've already used them
        'RoomService_Capped', 'FoodCourt_Capped', 'ShoppingMall_Capped',
        'Spa_Capped', 'VRDeck_Capped', 'Age_Capped'
    ]

    final_df = df.drop(columns=[col for col in drop_cols if col in df.columns])

    # Convert any remaining object columns to numeric
    for col in final_df.columns:
        if final_df[col].dtype == 'object' and col != 'PassengerId':
            le = LabelEncoder()
            final_df[col] = le.fit_transform(final_df[col].astype(str))

    # Final cleanup
    final_df = final_df.fillna(0)

    print(f"   → Final dataset shape: {final_df.shape}")
    print(f"   → Total features: {final_df.shape[1] - (1 if 'Transported' in final_df.columns else 0)}")
    print(f"   → Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    return final_df

def run_feature_engineering_pipeline(df):
    """FIXED Feature Engineering Pipeline - No Target Leakage"""
    print("🚀 FIXED SPACESHIP TITANIC FEATURE ENGINEERING PIPELINE")
    print("=" * 60)
    print("🔧 FIXES:")
    print("   - Removed all target-derived features")
    print("   - Removed target encoding")
    print("   - Reduced complex interactions")
    print("   - Added regularization-friendly features")
    print("=" * 60)

    # Make an explicit copy and preserve PassengerId early
    df = df.copy()
    df['PassengerId'] = df['PassengerId'].astype(str)
    df['_PassengerId_Original'] = df['PassengerId']

    original_features = df.shape[1]

    # Step 1: Basic preprocessing
    df = basic_missing_value_imputation(df)
    df = detect_and_handle_outliers(df)

    # Step 2: Core feature engineering (FIXED)
    df = engineer_passenger_groups(df)
    df = advanced_missing_value_imputation(df)
    df = engineer_cabin_features(df)
    df = engineer_spending_features(df)
    df = engineer_demographic_features(df)
    df = engineer_family_features(df)

    # Step 3: Safe advanced features
    df = create_safe_interaction_features(df)
    df = safe_categorical_encoding(df)

    # Step 4: Final dataset
    final_df = create_final_feature_set(df)

    print("\n" + "=" * 60)
    print("🎉 FIXED FEATURE ENGINEERING COMPLETE!")
    print(f"   📊 Original features: {original_features}")
    print(f"   📊 Final features: {final_df.shape[1] - (1 if 'Transported' in final_df.columns else 0)}")
    print(f"   📊 Feature expansion: {((final_df.shape[1] - 1) / original_features * 100):.1f}%")
    print("\n🔧 KEY FIXES APPLIED:")
    print("   ✅ Removed GroupSurvivalRate (target leakage)")
    print("   ✅ Removed FamilySurvivalRate (target leakage)")
    print("   ✅ Removed PlanetSurvivalRate (target leakage)")
    print("   ✅ Removed DestSurvivalRate (target leakage)")
    print("   ✅ Removed Target Encoding (major leakage source)")
    print("   ✅ Reduced complex 3-way interactions")
    print("   ✅ Added frequency-based safe features")
    print("=" * 60)

    return final_df

# Additional utility function for cross-validation target encoding (if needed)
def safe_target_encoding_cv(df, categorical_col, target_col, n_folds=5, smoothing=10):
    """
    Performs safe target encoding using cross-validation to prevent leakage.
    Use this ONLY if you really need target encoding features.
    """
    print(f"🔒 Safe Target Encoding for {categorical_col} using {n_folds}-fold CV...")

    encoded_values = np.zeros(len(df))
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    for train_idx, val_idx in kf.split(df):
        # Calculate target mean only on training fold
        train_df = df.iloc[train_idx]
        target_mean = train_df.groupby(categorical_col)[target_col].mean()
        global_mean = train_df[target_col].mean()
        category_counts = train_df.groupby(categorical_col).size()

        # Apply smoothing to prevent overfitting
        smoothed_means = (target_mean * category_counts + global_mean * smoothing) / (category_counts + smoothing)

        # Encode validation fold
        encoded_values[val_idx] = df.iloc[val_idx][categorical_col].map(smoothed_means).fillna(global_mean)

    return encoded_values

In [10]:
# 1. Load the raw dataset
df_train = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/spaceship_titanic_train.csv")

# 2. Run the full feature engineering pipeline (ensure the function is defined)
df_processed_train = run_feature_engineering_pipeline(df_train)

# 3: Export the complete processed dataframe
df_processed_train.to_csv('train_dataset_spaceship_titanic_processed.csv', index=False)
print(f"   ✅ Complete dataset exported: train_dataset_spaceship_titanic_processed.csv")
print(f"   📊 Shape: {df_processed_train.shape}")

🚀 FIXED SPACESHIP TITANIC FEATURE ENGINEERING PIPELINE
🔧 FIXES:
   - Removed all target-derived features
   - Removed target encoding
   - Reduced complex interactions
   - Added regularization-friendly features
🔧 Basic Missing Value Imputation...
   → Remaining missing values: 200
📊 Outlier Detection & Treatment...
   → RoomService: 1906 outliers capped
   → FoodCourt: 1916 outliers capped
   → ShoppingMall: 1879 outliers capped
   → Spa: 1833 outliers capped
   → VRDeck: 1849 outliers capped
   → Age outliers: 0
   → Spending outliers flagged: 408
🔍 Engineering Passenger Group Features...
   → Group sizes: 1-8
   → Solo travelers: 4,805 (55.3%)
   ✅ FIXED: Removed GroupSurvivalRate (target leakage)
🔧 Advanced Missing Value Imputation...
   → Applied hierarchical age imputation
🏢 Engineering Cabin Features...
   → Decks found: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'Unknown']
   → Starboard preference: 49.3%
💰 Engineering Spending Features...
   → Non-spenders: 3,653 (42.0%)
   → Hi

In [11]:
# 1. Load the raw dataset for Test set
df_test = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/refs/heads/main/data/spaceship_titanic_test.csv")

# 2. Run the full feature engineering pipeline (ensure the function is defined)
df_processed_test = run_feature_engineering_pipeline(df_test)

# 3: Export the complete processed dataframe
df_processed_test.to_csv('test_dataset_spaceship_titanic_processed.csv', index=False)
print(f"   ✅ Complete dataset exported: test_dataset_spaceship_titanic_processed.csv")
print(f"   📊 Shape: {df_processed_test.shape}")

🚀 FIXED SPACESHIP TITANIC FEATURE ENGINEERING PIPELINE
🔧 FIXES:
   - Removed all target-derived features
   - Removed target encoding
   - Reduced complex interactions
   - Added regularization-friendly features
🔧 Basic Missing Value Imputation...
   → Remaining missing values: 94
📊 Outlier Detection & Treatment...
   → RoomService: 919 outliers capped
   → FoodCourt: 931 outliers capped
   → ShoppingMall: 912 outliers capped
   → Spa: 921 outliers capped
   → VRDeck: 927 outliers capped
   → Age outliers: 0
   → Spending outliers flagged: 193
🔍 Engineering Passenger Group Features...
   → Group sizes: 1-8
   → Solo travelers: 2,340 (54.7%)
   ✅ FIXED: Removed GroupSurvivalRate (target leakage)
🔧 Advanced Missing Value Imputation...
   → Applied hierarchical age imputation
🏢 Engineering Cabin Features...
   → Decks found: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'Unknown']
   → Starboard preference: 48.9%
💰 Engineering Spending Features...
   → Non-spenders: 1,804 (42.2%)
   → High spe

In [12]:
df_processed_test.head(5)

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HasAgeOutlier,...,CabinDeck_G,CabinDeck_T,CabinDeck_Unknown,CabinSide_S,CabinSide_U,FirstNameLength,LastNameLength,FullNameLength,HasCommonFirstName,HasRareLastName
0,0013_01,1,27.0,0,0.0,0.0,0.0,0.0,0.0,0,...,True,False,False,True,False,5.0,9.0,14.0,False,False
1,0018_01,0,19.0,0,0.0,9.0,0.0,107.5,0.0,0,...,False,False,False,True,False,6.0,7.0,13.0,False,True
2,0019_01,1,31.0,0,0.0,0.0,0.0,0.0,0.0,0,...,False,False,False,True,False,5.0,9.0,14.0,False,True
3,0021_01,0,38.0,0,0.0,165.0,0.0,107.5,77.5,0,...,False,False,False,True,False,6.0,9.0,15.0,False,True
4,0023_01,0,20.0,0,10.0,0.0,67.5,0.0,0.0,0,...,False,False,False,True,False,6.0,8.0,14.0,False,False
